In [1]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

# Configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784  # 28x28
num_epochs = 200
batch_size = 100
sample_dir = 'sample'

# Create a directory if not exits
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
    
# Image processing
# transform = transforms.Compose([
#     transforms.ToTensor(), 
#     transforms.Normalize(mean=(0.5, 0.5, 0.5),  # 3 channels for RGB
#                          std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

# mnist dataset
mnist = torchvision.datasets.MNIST(root='../data', 
                                   train=True, 
                                   transform=transform, 
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist, 
                                          batch_size=batch_size, 
                                          shuffle=True)

$$ReLU(x)=max(0,x)$$

$$LeakyReLU(x)=max(0,x)+negative\_slope\times min(0,x)$$

In [3]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size), 
    nn.LeakyReLU(0.2), 
    nn.Linear(hidden_size, hidden_size), 
    nn.LeakyReLU(0.2), 
    nn.Linear(hidden_size, 1), 
    nn.Sigmoid())

# Generator
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size), 
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size), 
    nn.ReLU(), 
    nn.Linear(hidden_size, image_size), 
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

# Reverse normalization
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [4]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(-1, 28*28).to(device)
        
        # Create the labels which are later used as input for BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        
        # Compute BCE_Loss using real images where BCE_Loss(x, y): -y*log(D(x))-(1-y)*log(1-D(x))
        # Second term of the loss is always zero since real_label == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_label == 1
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #        
        
        # Compute loss for fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D{G{z}}) instead of minimizing log(1-D{G{z}})
        # For the reason, see the last paragraph of section 3, https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, D(x): {:.2f} g_loss: {:.4f}, G(x): {:.2f}'
                  .format(epoch+1, num_epochs, i+1, total_step, d_loss.item(), real_score.mean().item(), 
                          g_loss.item(), fake_score.mean().item()))
    
    # Save real image
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)  # (batch_size, 1, w, h)
        save_image(denorm(images), os.path.join(sample_dir, 'real_image.png'))
        
    # Save sampled images each epoch
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    
# Save the model checkpoint
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [1/200], Step [100/600], d_loss: 0.3210, D(x): 0.92 g_loss: 2.2411, G(x): 0.21
Epoch [1/200], Step [200/600], d_loss: 0.0448, D(x): 0.99 g_loss: 4.1805, G(x): 0.04
Epoch [1/200], Step [300/600], d_loss: 0.1366, D(x): 0.95 g_loss: 3.5445, G(x): 0.08
Epoch [1/200], Step [400/600], d_loss: 0.1452, D(x): 0.96 g_loss: 5.5637, G(x): 0.03
Epoch [1/200], Step [500/600], d_loss: 0.0195, D(x): 1.00 g_loss: 5.1947, G(x): 0.02
Epoch [1/200], Step [600/600], d_loss: 0.0804, D(x): 0.97 g_loss: 4.6233, G(x): 0.05
Epoch [2/200], Step [100/600], d_loss: 0.1566, D(x): 0.94 g_loss: 3.6037, G(x): 0.08
Epoch [2/200], Step [200/600], d_loss: 0.0293, D(x): 1.00 g_loss: 5.7622, G(x): 0.03
Epoch [2/200], Step [300/600], d_loss: 0.0525, D(x): 0.97 g_loss: 5.0017, G(x): 0.02
Epoch [2/200], Step [400/600], d_loss: 0.4968, D(x): 0.82 g_loss: 4.0269, G(x): 0.14
Epoch [2/200], Step [500/600], d_loss: 0.4095, D(x): 0.88 g_loss: 4.8030, G(x): 0.13
Epoch [2/200], Step [600/600], d_loss: 0.0388, D(x): 0.99 g_loss:

Epoch [17/200], Step [100/600], d_loss: 0.2197, D(x): 0.94 g_loss: 4.2370, G(x): 0.08
Epoch [17/200], Step [200/600], d_loss: 0.2885, D(x): 0.90 g_loss: 6.6604, G(x): 0.01
Epoch [17/200], Step [300/600], d_loss: 0.2565, D(x): 0.93 g_loss: 4.9438, G(x): 0.06
Epoch [17/200], Step [400/600], d_loss: 0.2313, D(x): 0.91 g_loss: 5.0032, G(x): 0.01
Epoch [17/200], Step [500/600], d_loss: 0.2174, D(x): 0.95 g_loss: 5.2533, G(x): 0.08
Epoch [17/200], Step [600/600], d_loss: 0.1774, D(x): 0.92 g_loss: 5.8976, G(x): 0.03
Epoch [18/200], Step [100/600], d_loss: 0.3175, D(x): 0.96 g_loss: 5.3761, G(x): 0.15
Epoch [18/200], Step [200/600], d_loss: 0.1700, D(x): 0.94 g_loss: 4.6174, G(x): 0.06
Epoch [18/200], Step [300/600], d_loss: 0.2821, D(x): 0.96 g_loss: 3.6653, G(x): 0.15
Epoch [18/200], Step [400/600], d_loss: 0.1944, D(x): 0.97 g_loss: 5.8170, G(x): 0.10
Epoch [18/200], Step [500/600], d_loss: 0.2171, D(x): 0.95 g_loss: 6.1448, G(x): 0.09
Epoch [18/200], Step [600/600], d_loss: 0.7631, D(x): 

Epoch [33/200], Step [100/600], d_loss: 0.6442, D(x): 0.91 g_loss: 2.4788, G(x): 0.31
Epoch [33/200], Step [200/600], d_loss: 0.3051, D(x): 0.91 g_loss: 3.9658, G(x): 0.12
Epoch [33/200], Step [300/600], d_loss: 0.3501, D(x): 0.90 g_loss: 4.7889, G(x): 0.09
Epoch [33/200], Step [400/600], d_loss: 0.4909, D(x): 0.97 g_loss: 4.1358, G(x): 0.21
Epoch [33/200], Step [500/600], d_loss: 0.4846, D(x): 0.81 g_loss: 3.8170, G(x): 0.06
Epoch [33/200], Step [600/600], d_loss: 0.6888, D(x): 0.86 g_loss: 3.2066, G(x): 0.20
Epoch [34/200], Step [100/600], d_loss: 0.2436, D(x): 0.98 g_loss: 4.2414, G(x): 0.15
Epoch [34/200], Step [200/600], d_loss: 0.4320, D(x): 0.91 g_loss: 4.5810, G(x): 0.17
Epoch [34/200], Step [300/600], d_loss: 0.3896, D(x): 0.88 g_loss: 4.7848, G(x): 0.09
Epoch [34/200], Step [400/600], d_loss: 0.4127, D(x): 0.92 g_loss: 3.7672, G(x): 0.15
Epoch [34/200], Step [500/600], d_loss: 0.3070, D(x): 0.88 g_loss: 4.8993, G(x): 0.05
Epoch [34/200], Step [600/600], d_loss: 0.4824, D(x): 

Epoch [49/200], Step [100/600], d_loss: 0.7047, D(x): 0.73 g_loss: 1.9624, G(x): 0.13
Epoch [49/200], Step [200/600], d_loss: 0.7016, D(x): 0.79 g_loss: 2.4957, G(x): 0.22
Epoch [49/200], Step [300/600], d_loss: 0.5790, D(x): 0.86 g_loss: 2.5881, G(x): 0.26
Epoch [49/200], Step [400/600], d_loss: 0.6072, D(x): 0.83 g_loss: 2.1002, G(x): 0.22
Epoch [49/200], Step [500/600], d_loss: 0.4412, D(x): 0.87 g_loss: 3.0408, G(x): 0.17
Epoch [49/200], Step [600/600], d_loss: 0.8347, D(x): 0.74 g_loss: 2.4124, G(x): 0.20
Epoch [50/200], Step [100/600], d_loss: 0.7058, D(x): 0.84 g_loss: 2.1668, G(x): 0.27
Epoch [50/200], Step [200/600], d_loss: 0.4103, D(x): 0.80 g_loss: 2.4846, G(x): 0.10
Epoch [50/200], Step [300/600], d_loss: 0.6557, D(x): 0.86 g_loss: 3.6477, G(x): 0.28
Epoch [50/200], Step [400/600], d_loss: 0.5505, D(x): 0.83 g_loss: 2.8144, G(x): 0.18
Epoch [50/200], Step [500/600], d_loss: 0.5804, D(x): 0.86 g_loss: 3.0284, G(x): 0.24
Epoch [50/200], Step [600/600], d_loss: 0.7650, D(x): 

Epoch [65/200], Step [100/600], d_loss: 0.6777, D(x): 0.81 g_loss: 2.4530, G(x): 0.26
Epoch [65/200], Step [200/600], d_loss: 0.4917, D(x): 0.84 g_loss: 2.6055, G(x): 0.17
Epoch [65/200], Step [300/600], d_loss: 0.7002, D(x): 0.87 g_loss: 2.2328, G(x): 0.30
Epoch [65/200], Step [400/600], d_loss: 0.5483, D(x): 0.88 g_loss: 2.7281, G(x): 0.24
Epoch [65/200], Step [500/600], d_loss: 0.7087, D(x): 0.83 g_loss: 2.1551, G(x): 0.26
Epoch [65/200], Step [600/600], d_loss: 0.6387, D(x): 0.82 g_loss: 2.3115, G(x): 0.23
Epoch [66/200], Step [100/600], d_loss: 0.5806, D(x): 0.77 g_loss: 2.5863, G(x): 0.15
Epoch [66/200], Step [200/600], d_loss: 0.4844, D(x): 0.79 g_loss: 2.4653, G(x): 0.11
Epoch [66/200], Step [300/600], d_loss: 0.6164, D(x): 0.80 g_loss: 2.7871, G(x): 0.21
Epoch [66/200], Step [400/600], d_loss: 0.6736, D(x): 0.85 g_loss: 2.2215, G(x): 0.29
Epoch [66/200], Step [500/600], d_loss: 0.3928, D(x): 0.85 g_loss: 2.8095, G(x): 0.12
Epoch [66/200], Step [600/600], d_loss: 0.4994, D(x): 

...

Epoch [128/200], Step [500/600], d_loss: 1.0326, D(x): 0.67 g_loss: 1.6089, G(x): 0.29
Epoch [128/200], Step [600/600], d_loss: 0.9570, D(x): 0.68 g_loss: 1.7121, G(x): 0.29
Epoch [129/200], Step [100/600], d_loss: 0.8820, D(x): 0.71 g_loss: 1.8942, G(x): 0.26
Epoch [129/200], Step [200/600], d_loss: 0.7697, D(x): 0.69 g_loss: 1.9271, G(x): 0.22
Epoch [129/200], Step [300/600], d_loss: 1.1035, D(x): 0.57 g_loss: 1.6014, G(x): 0.22
Epoch [129/200], Step [400/600], d_loss: 1.0141, D(x): 0.65 g_loss: 1.6232, G(x): 0.30
Epoch [129/200], Step [500/600], d_loss: 0.9534, D(x): 0.64 g_loss: 1.4146, G(x): 0.24
Epoch [129/200], Step [600/600], d_loss: 0.7529, D(x): 0.72 g_loss: 1.6009, G(x): 0.24
Epoch [130/200], Step [100/600], d_loss: 0.7712, D(x): 0.75 g_loss: 1.7184, G(x): 0.28
Epoch [130/200], Step [200/600], d_loss: 0.8856, D(x): 0.70 g_loss: 1.8834, G(x): 0.26
Epoch [130/200], Step [300/600], d_loss: 0.7820, D(x): 0.80 g_loss: 1.6559, G(x): 0.33
Epoch [130/200], Step [400/600], d_loss: 1.

Epoch [144/200], Step [400/600], d_loss: 0.8158, D(x): 0.75 g_loss: 1.5521, G(x): 0.31
Epoch [144/200], Step [500/600], d_loss: 1.0606, D(x): 0.71 g_loss: 1.8938, G(x): 0.36
Epoch [144/200], Step [600/600], d_loss: 0.7867, D(x): 0.75 g_loss: 2.0301, G(x): 0.27
Epoch [145/200], Step [100/600], d_loss: 0.8057, D(x): 0.76 g_loss: 1.6557, G(x): 0.31
Epoch [145/200], Step [200/600], d_loss: 0.8934, D(x): 0.73 g_loss: 1.4531, G(x): 0.31
Epoch [145/200], Step [300/600], d_loss: 0.8653, D(x): 0.68 g_loss: 1.6434, G(x): 0.27
Epoch [145/200], Step [400/600], d_loss: 0.8382, D(x): 0.73 g_loss: 1.4728, G(x): 0.29
Epoch [145/200], Step [500/600], d_loss: 0.9035, D(x): 0.67 g_loss: 1.5837, G(x): 0.27
Epoch [145/200], Step [600/600], d_loss: 1.1725, D(x): 0.56 g_loss: 1.5744, G(x): 0.28
Epoch [146/200], Step [100/600], d_loss: 0.7027, D(x): 0.76 g_loss: 1.7776, G(x): 0.24
Epoch [146/200], Step [200/600], d_loss: 1.0867, D(x): 0.70 g_loss: 1.4803, G(x): 0.37
Epoch [146/200], Step [300/600], d_loss: 1.

Epoch [160/200], Step [300/600], d_loss: 1.0485, D(x): 0.70 g_loss: 1.8496, G(x): 0.38
Epoch [160/200], Step [400/600], d_loss: 0.9653, D(x): 0.69 g_loss: 1.5650, G(x): 0.28
Epoch [160/200], Step [500/600], d_loss: 0.8998, D(x): 0.66 g_loss: 1.3561, G(x): 0.27
Epoch [160/200], Step [600/600], d_loss: 0.8295, D(x): 0.70 g_loss: 1.3814, G(x): 0.26
Epoch [161/200], Step [100/600], d_loss: 0.8958, D(x): 0.66 g_loss: 1.8131, G(x): 0.27
Epoch [161/200], Step [200/600], d_loss: 0.9285, D(x): 0.74 g_loss: 1.8837, G(x): 0.36
Epoch [161/200], Step [300/600], d_loss: 0.7760, D(x): 0.73 g_loss: 1.5073, G(x): 0.28
Epoch [161/200], Step [400/600], d_loss: 1.0661, D(x): 0.66 g_loss: 1.5984, G(x): 0.34
Epoch [161/200], Step [500/600], d_loss: 0.9596, D(x): 0.64 g_loss: 1.4739, G(x): 0.27
Epoch [161/200], Step [600/600], d_loss: 1.0498, D(x): 0.73 g_loss: 1.4462, G(x): 0.38
Epoch [162/200], Step [100/600], d_loss: 1.1934, D(x): 0.72 g_loss: 1.3813, G(x): 0.42
Epoch [162/200], Step [200/600], d_loss: 0.

Epoch [176/200], Step [200/600], d_loss: 1.2385, D(x): 0.71 g_loss: 1.2404, G(x): 0.45
Epoch [176/200], Step [300/600], d_loss: 1.0076, D(x): 0.67 g_loss: 1.3345, G(x): 0.34
Epoch [176/200], Step [400/600], d_loss: 0.8094, D(x): 0.75 g_loss: 1.3106, G(x): 0.31
Epoch [176/200], Step [500/600], d_loss: 1.0967, D(x): 0.62 g_loss: 1.0440, G(x): 0.32
Epoch [176/200], Step [600/600], d_loss: 0.9182, D(x): 0.67 g_loss: 1.6018, G(x): 0.28
Epoch [177/200], Step [100/600], d_loss: 0.8669, D(x): 0.75 g_loss: 1.5656, G(x): 0.34
Epoch [177/200], Step [200/600], d_loss: 0.8586, D(x): 0.73 g_loss: 1.2098, G(x): 0.32
Epoch [177/200], Step [300/600], d_loss: 0.9334, D(x): 0.67 g_loss: 1.6005, G(x): 0.28
Epoch [177/200], Step [400/600], d_loss: 1.0214, D(x): 0.74 g_loss: 1.4935, G(x): 0.40
Epoch [177/200], Step [500/600], d_loss: 0.9264, D(x): 0.68 g_loss: 1.7151, G(x): 0.30
Epoch [177/200], Step [600/600], d_loss: 1.0388, D(x): 0.69 g_loss: 1.3035, G(x): 0.35
Epoch [178/200], Step [100/600], d_loss: 0.

Epoch [192/200], Step [100/600], d_loss: 0.8826, D(x): 0.72 g_loss: 1.7953, G(x): 0.31
Epoch [192/200], Step [200/600], d_loss: 1.0586, D(x): 0.69 g_loss: 1.2839, G(x): 0.37
Epoch [192/200], Step [300/600], d_loss: 1.0338, D(x): 0.65 g_loss: 1.5712, G(x): 0.30
Epoch [192/200], Step [400/600], d_loss: 0.8150, D(x): 0.74 g_loss: 1.6153, G(x): 0.31
Epoch [192/200], Step [500/600], d_loss: 0.8643, D(x): 0.71 g_loss: 1.8776, G(x): 0.28
Epoch [192/200], Step [600/600], d_loss: 0.9116, D(x): 0.69 g_loss: 1.3624, G(x): 0.31
Epoch [193/200], Step [100/600], d_loss: 0.9988, D(x): 0.65 g_loss: 1.5284, G(x): 0.32
Epoch [193/200], Step [200/600], d_loss: 0.8947, D(x): 0.63 g_loss: 1.9237, G(x): 0.22
Epoch [193/200], Step [300/600], d_loss: 0.9475, D(x): 0.71 g_loss: 1.5844, G(x): 0.32
Epoch [193/200], Step [400/600], d_loss: 0.9412, D(x): 0.66 g_loss: 1.1963, G(x): 0.27
Epoch [193/200], Step [500/600], d_loss: 1.0684, D(x): 0.63 g_loss: 1.6020, G(x): 0.31
Epoch [193/200], Step [600/600], d_loss: 1.